
# RoamRhythm — FIT → GPX → POIs → SSML (Interactive Notebook)

This notebook provides an **interactive**, well-documented workflow to:
- Convert **`.FIT` → `.GPX`**
- Generate **evenly spaced POIs** over distance
- Produce **calm/enthusiastic SSML** (intro, per-POI, outro)

> **Stack**: Conda (Python 3.13.5), `fitdecode` or `fitparse`, `gpxpy`, `numpy`  
> **Workflow**: iterate here → migrate stable logic back to a Python CLI/script later.



## 0) Environment & dependencies (Conda + pip)

Create the environment once, then run Jupyter from it:

```bash
conda create -n roamrhythm python=3.13.5 -y
conda activate roamrhythm
pip install --upgrade pip
pip install jupyterlab nbformat fitdecode fitparse gpxpy numpy
```
> You only need **one** of `fitdecode` or `fitparse`. Installing both increases compatibility.



## 1) Project config (auto locate your FIT file)

- Set `FIT_HINT` to either an **absolute path** or just the **filename**.  
- If only a filename is given, we **auto-search** recursively from the notebook folder.


In [1]:

from pathlib import Path
import sys

# --- Configure either a direct path OR just a filename to auto-search ---
FIT_HINT = r".\Data\Test10K_BE_UD.fit"   # change to your file name or absolute path
OUTDIR   = Path("./out")
PACE_KMH = 4.0
POI_COUNT = 10
USE_FITDECODE = True  # set False to try fitparse

def resolve_fit_path(hint: str) -> Path:
    p = Path(hint)
    if p.exists():
        return p.resolve()
    here = Path.cwd()
    candidate = here / hint
    if candidate.exists():
        return candidate.resolve()
    matches = list(here.rglob(hint if "*" in hint else f"*{Path(hint).stem}*.fit"))
    if matches:
        return matches[0].resolve()
    raise FileNotFoundError(
        f"FIT file not found.\n"
        f"Tried: {p}\n"
        f"CWD : {here}\n"
        f"Tips:\n"
        f" - Put the .fit next to the notebook OR give an absolute path.\n"
        f" - On Windows, use raw strings: r\"C:\\path\\to\\file.fit\".\n"
        f" - Check extension case (.fit vs .FIT).\n"
    )

FIT_PATH = resolve_fit_path(FIT_HINT)
OUTDIR.mkdir(parents=True, exist_ok=True)
print("Using FIT:", FIT_PATH)
print("Output dir:", OUTDIR.resolve())


Using FIT: C:\Development\Python\RoamRhythm\data\Test10K_BE_UD.fit
Output dir: C:\Development\Python\RoamRhythm\out



## 2) Imports & helpers (Optional)


In [ ]:
%pip install nbformat fitdecode fitparse gpxpy numpy

In [2]:

import math, csv, json
from datetime import datetime, timedelta, timezone
from typing import List, Tuple, Optional

import numpy as np
import gpxpy, gpxpy.gpx

# Try selected FIT backend first; fall back to the other
fit_backend = None
if USE_FITDECODE:
    try:
        import fitdecode  # type: ignore
        fit_backend = "fitdecode"
    except Exception as e:
        print("fitdecode not available:", e)
        fit_backend = None

if fit_backend is None:
    try:
        from fitparse import FitFile  # type: ignore
        fit_backend = "fitparse"
    except Exception as e:
        print("fitparse not available:", e)
        raise RuntimeError("No FIT backend available. Please install fitdecode or fitparse.")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c  # meters

def cumulative_distance(points: List[Tuple[float,float]]) -> List[float]:
    dists = [0.0]
    for i in range(1, len(points)):
        d = haversine(points[i-1][0], points[i-1][1], points[i][0], points[i][1])
        dists.append(dists[-1] + d)
    return dists

def read_fit_fitdecode(path: Path):
    lats, lons, eles, times = [], [], [], []
    with fitdecode.FitReader(str(path)) as rd:
        for frame in rd:
            if isinstance(frame, fitdecode.records.FitDataMessage) and frame.name == 'record':
                lat = frame.get_value('position_lat')
                lon = frame.get_value('position_long')
                ele = frame.get_value('enhanced_altitude')
                t = frame.get_value('timestamp')
                def semi_to_deg(x): return x * (180.0 / 2**31) if x is not None else None
                lat = semi_to_deg(lat); lon = semi_to_deg(lon)
                if lat is not None and lon is not None:
                    lats.append(lat); lons.append(lon); eles.append(ele if ele is not None else 0.0)
                    times.append(t if isinstance(t, datetime) else None)
    return lats, lons, eles, times

def read_fit_fitparse(path: Path):
    lats, lons, eles, times = [], [], [], []
    ff = FitFile(str(path))
    for record in ff.get_messages('record'):
        vals = {d.name: d.value for d in record}
        lat = vals.get('position_lat'); lon = vals.get('position_long')
        ele = vals.get('enhanced_altitude', 0.0); t = vals.get('timestamp', None)
        def semi_to_deg(x): return x * (180.0 / 2**31) if x is not None else None
        lat = semi_to_deg(lat); lon = semi_to_deg(lon)
        if lat is not None and lon is not None:
            lats.append(lat); lons.append(lon); eles.append(ele if ele is not None else 0.0)
            times.append(t if isinstance(t, datetime) else None)
    return lats, lons, eles, times

def fit_to_track(path: Path):
    if fit_backend == "fitdecode":
        return read_fit_fitdecode(path)
    elif fit_backend == "fitparse":
        return read_fit_fitparse(path)
    else:
        raise RuntimeError("No FIT backend selected.")

def make_gpx(lats, lons, eles, times, creator="RoamRhythm-notebook"):
    gpx = gpxpy.gpx.GPX()
    gpx.creator = creator
    trk = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(trk)
    seg = gpxpy.gpx.GPXTrackSegment()
    trk.segments.append(seg)
    for lat, lon, ele, t in zip(lats, lons, eles, times):
        seg.points.append(gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele, time=t))
    return gpx

def interpolate_time_from_pace(start_time: Optional[datetime], dist_m: float, pace_kmh: float) -> datetime:
    if start_time is None:
        start_time = datetime(1970,1,1, tzinfo=timezone.utc)
    hours = dist_m / (pace_kmh * 1000.0)
    return start_time + timedelta(hours=hours)

def ssml_wrap(text: str) -> str:
    return f'<speak><prosody rate="85%" pitch="+0st" volume="+0.0dB">{text}</prosody></speak>'

POI_TITLES = [
    "Landschap & waterstaat","Ontstaan van nederzettingen","Ambachten en handel","Architectuur in de regio",
    "Natuur & biodiversiteit","Polders, dijken en gemalen","Oude routes & verbindingen","Oorlogssporen en herdenking",
    "Bestuur & gemeentefusies","Moderne tijd & toekomst"
]
POI_TEMPLATES = [
    "Dit landschap is gevormd door water en wind. Let op sloten, kades en hoogtes: ze vertellen hoe mensen het water sturen. Een klein weetje: veel polderwegen liggen net hoger dan omliggende velden.",
    "Veel dorpen ontstonden bij kruisingen van wegen en water. Luister hoe handel en ambacht bewoners trokken—vaak rond een kerk of plein als hart van de gemeenschap.",
    "Ambachten bloeiden waar grondstoffen en routes elkaar raken. Denk aan molenaars, smeden, scheepstimmerlieden of linnenwevers—ieder gebonden aan seizoen en plaats.",
    "Kijk naar metselwerk, dakvormen en gevelopeningen. Baksteen en nuchtere lijnen domineren vaak; wederopbouw en modernisering lieten ook hun sporen na.",
    "Let op vogels aan randen van water en riet. Fluisterstil bewegen vaak waterhoentjes; in sloten zie je soms libellen. Tip: loop even langzamer en luister naar lagen van geluid.",
    "Polderen is samenwerken: dijken houden water buiten, sloten voeren af, gemalen regelen peil. Een metafoor: het landschap ademt—in natte tijden trager, in droge sneller.",
    "Routes vormen ritme: karrenpaden, trekvaarten, kanalen of spoorlijnen verplaatsen mensen en ideeën. Oude tracés wonen soms voort als stille fietspaden.",
    "Sporen van oorlog vragen bedachtzaamheid. Monumenten zijn ankerpunten van herinnering; ze nodigen uit tot stilstaan, kijken, en zachtjes verdergaan.",
    "Gemeenten veranderden mee met tijd en taken. Fusies bundelden diensten; grenzen verschoof men om samen sterker te staan in zorg, onderwijs en beheer.",
    "Vandaag draait veel om balans: natuurontwikkeling, energie, wonen en recreatie. Kijk om je heen: vernieuwing en behoud kunnen elkaar versterken."
]



## 3) Load `.FIT` → write `.GPX`


In [3]:

lats, lons, eles, times = fit_to_track(FIT_PATH)
if len(lats) < 2:
    raise RuntimeError("Not enough track points in FIT.")
gpx = make_gpx(lats, lons, eles, times)
gpx_path = OUTDIR / (FIT_PATH.stem + ".gpx")
gpx_path.write_text(gpx.to_xml(), encoding="utf-8")
print(f"GPX written: {gpx_path.resolve()}")
print(f"Track points: {len(lats)}")


GPX written: C:\Development\Python\RoamRhythm\out\Test10K_BE_UD.gpx
Track points: 1196



## 4) Compute distance & build evenly-spaced POIs


In [4]:

points = list(zip(lats, lons))
cum = cumulative_distance(points)  # meters
total_m = cum[-1]
print(f"Total distance: {total_m/1000.0:.2f} km")

# targets from 9% to 99% of route (avoid overlap with intro/outro)
targets = np.linspace(total_m * 0.09, total_m * 0.99, POI_COUNT)

# find first timestamp (if any)
t0 = next((t for t in times if isinstance(t, datetime)), None)

pois = []
for i, tm in enumerate(targets):
    idx = int(np.argmin([abs(d - tm) for d in cum]))
    km = round(cum[idx]/1000.0, 2)
    lat, lon = float(lats[idx]), float(lons[idx])
    eta = times[idx] if (idx < len(times) and isinstance(times[idx], datetime)) else None
    if eta is None:
        eta = (t0 if t0 else datetime(1970,1,1, tzinfo=timezone.utc))
        hours = (cum[idx] / 1000.0) / PACE_KMH
        eta = eta + timedelta(hours=hours)
    dur_s = 100 if i % 2 == 0 else 95
    title = POI_TITLES[i % len(POI_TITLES)]
    text  = POI_TEMPLATES[i % len(POI_TEMPLATES)]
    ssml  = ssml_wrap(f"{title}. {text}")
    pois.append({
        "id": f"POI-{i+1:02d}",
        "km": km, "lat": lat, "lon": lon,
        "eta_utc": eta.astimezone(timezone.utc).isoformat(),
        "duration_s": dur_s, "title": title, "ssml": ssml
    })

print(f"POIs generated: {len(pois)}")


Total distance: 10.14 km
POIs generated: 10



## 5) Intro & Outro SSML


In [5]:

intro_text = ("Welkom bij deze wandeling. We nemen u in een rustig tempo mee door het landschap, "
              "met korte verhalen over de omgeving, afgewisseld met muziek uit de jaren tachtig, "
              "taalprikkels Noors en ontspannen stretchmomenten. "
              "Wandel op uw gemak, kijk om u heen en luister wanneer we een interessant punt passeren. "
              "Aan het begin starten we met een korte warming-up. Klaar? Dan gaan we op pad.")
outro_text = ("We zijn bijna aan het einde van de route. Neem even de tijd om na te voelen hoe uw lichaam is "
              "opgewarmd. Straks sluiten we af met enkele rustige stretches. "
              "Dank voor het meewandelen. Heeft u iets nieuws ontdekt of geleerd—een woord Noors, een stukje "
              "geschiedenis? Bewaar het gevoel van ruimte en ritme. Tot de volgende keer, en fijne dag verder.")
intro_ssml = ssml_wrap(intro_text)
outro_ssml = ssml_wrap(outro_text)



## 6) Write artifacts (GPX, POI CSV, SSML files, Manifest)


In [6]:

ssml_dir = OUTDIR / "ssml"
ssml_dir.mkdir(exist_ok=True)

# Save SSML
(ssml_dir / "intro.ssml").write_text(intro_ssml, encoding="utf-8")
(ssml_dir / "outro.ssml").write_text(outro_ssml, encoding="utf-8")
for poi in pois:
    (ssml_dir / f"{poi['id']}.ssml").write_text(poi["ssml"], encoding="utf-8")

# Save CSV
csv_path = OUTDIR / "pois.csv"
with csv_path.open("w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id","km","lat","lon","eta_utc","duration_s","title","ssml_file"])
    for poi in pois:
        w.writerow([poi["id"], poi["km"], poi["lat"], poi["lon"], poi["eta_utc"], poi["duration_s"], poi["title"], f"ssml/{poi['id']}.ssml"])

# Save Manifest
manifest = {
    "route": {
        "meters": round(float(cum[-1]), 1),
        "km": round(float(cum[-1])/1000.0, 3)
    },
    "gpx": (OUTDIR / (FIT_PATH.stem + ".gpx")).name,
    "pace_kmh": PACE_KMH,
    "ssml": {
        "intro": "ssml/intro.ssml",
        "outro": "ssml/outro.ssml"
    },
    "pois": [
        {
            "id": p["id"], "km": p["km"], "lat": p["lat"], "lon": p["lon"],
            "eta_utc": p["eta_utc"], "duration_s": p["duration_s"],
            "title": p["title"], "ssml": f"ssml/{p['id']}.ssml"
        } for p in pois
    ]
}
manifest_path = OUTDIR / "manifest.json"
manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")

print("Artifacts written:")
print(" - GPX:", gpx_path.resolve())
print(" - CSV:", csv_path.resolve())
print(" - SSML dir:", ssml_dir.resolve())
print(" - Manifest:", manifest_path.resolve())


Artifacts written:
 - GPX: C:\Development\Python\RoamRhythm\out\Test10K_BE_UD.gpx
 - CSV: C:\Development\Python\RoamRhythm\out\pois.csv
 - SSML dir: C:\Development\Python\RoamRhythm\out\ssml
 - Manifest: C:\Development\Python\RoamRhythm\out\manifest.json



## 7) Migration back to Python script

When you're happy with the workflow, extract functions from cells 2–6 into a module (e.g., `roamrhythm/core.py`) and add a small CLI (argparse) to create a script-friendly tool.



---
# Add-on: GPX → POIs → SSML → Offline Pack (integrated)
This section extends the notebook you use now. It builds a full **offline pack** from a GPX with your preferences, and can bundle everything into a ZIP for easy download/testing.



## A) Configuration
- `GPX_PATH`: your route (use the provided GPX or change path)
- `PACE_KMH`: fallback ETA if timestamps are missing
- `POI_COUNT`: points of interest count
- `PREFS`: categories, music share, language, brain training, stretches


In [9]:

from pathlib import Path
GPX_PATH  = Path(".\out\Test10K_BE_UD.gpx")  # change to your GPX if needed
OUTDIR    = Path("./pack_from_notebook")
PACE_KMH  = 4.0
POI_COUNT = 12

PREFS = {
    "categories": ["landschap","geschiedenis_ijzertijd","cultuur_bekende_personen","architectuur_route"],
    "depth": "standard",
    "music_share": 0.20,
    "language": {"code":"no-NO","deck":"lang/deck_no_A1.csv"},
    "brain": {"math": True, "memory": True},
    "stretches": True
}

# ensure output structure
OUTDIR.mkdir(parents=True, exist_ok=True)
for sub in ["ssml","gpx","cues","lang","brain"]:
    (OUTDIR/sub).mkdir(exist_ok=True)

print("GPX:", GPX_PATH.resolve())
print("OUT:", OUTDIR.resolve())


GPX: C:\Development\Python\RoamRhythm\out\Test10K_BE_UD.gpx
OUT: C:\Development\Python\RoamRhythm\pack_from_notebook



## B) Stdlib GPX parser + distance profile


In [10]:

import xml.etree.ElementTree as ET
from datetime import datetime, timedelta, timezone
import math, json, csv
import numpy as np

ns = {"g":"http://www.topografix.com/GPX/1/1"}
try:
    tree = ET.parse(GPX_PATH)
except ET.ParseError:
    ns = {}
    tree = ET.parse(GPX_PATH)
root = tree.getroot()

def findall(elem, path):
    res = elem.findall(path, ns)
    return res if res else elem.findall(path.replace("g:",""))

def find(elem, path):
    r = elem.find(path, ns)
    return r if r is not None else elem.find(path.replace("g:",""))

pts = []
for trk in findall(root, ".//g:trk") + findall(root, ".//trk"):
    for seg in findall(trk, ".//g:trkseg") + findall(trk, ".//trkseg"):
        for p in findall(seg, ".//g:trkpt") + findall(seg, ".//trkpt"):
            lat = float(p.attrib.get("lat")); lon = float(p.attrib.get("lon"))
            t_el = find(p,"g:time") or find(p,"time")
            t = datetime.fromisoformat(t_el.text.replace("Z","+00:00")) if (t_el is not None and t_el.text) else None
            pts.append((lat,lon,t))

assert len(pts) >= 2, "Not enough points in GPX."

def hav(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, atan2, sqrt
    R=6371000.0
    p1,p2=radians(lat1),radians(lat2)
    d1=radians(lat2-lat1); d2=radians(lon2-lon1)
    a=sin(d1/2)**2 + cos(p1)*cos(p2)*sin(d2/2)**2
    return 2*R*atan2(sqrt(a), sqrt(1-a))

cum=[0.0]
for i in range(1,len(pts)):
    cum.append(cum[-1] + hav(pts[i-1][0], pts[i-1][1], pts[i][0], pts[i][1]))

total_m = cum[-1]; total_km = total_m/1000.0
duration_min = int(round((total_km/ PACE_KMH) * 60))
print(f"Points: {len(pts)}  Distance: {total_km:.2f} km  Duration@4km/h: {duration_min} min")


Points: 1196  Distance: 10.14 km  Duration@4km/h: 152 min


C:\Users\charl\AppData\Local\Temp\ipykernel_43908\3638645266.py:27: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  t_el = find(p,"g:time") or find(p,"time")



## C) Category-specific SSML (calm, engaging; architecture is on-route only)


In [11]:

def ssml(text: str) -> str:
    return f'<speak><prosody rate="85%" pitch="+0st">{text}</prosody></speak>'

def text_for(cat: str) -> str:
    if cat == "landschap":
        return ("Landschap & waterstaat. Kijk naar sloten, kades en rietkragen; ze sturen het water. "
                "Luister heel even naar de lagen van geluid—wind, vogels, water—en wandel dan rustig verder.")
    if cat == "geschiedenis_ijzertijd":
        return ("Geschiedenis — focus op de IJzertijd. Nieuwe werktuigen en veranderende handelsroutes; "
                "nederzettingen kozen vaak een iets hogere, drogere ligging.")
    if cat == "cultuur_bekende_personen":
        return ("Cultuur & lokale figuren. Kleine anekdotes over ambachtslieden en dorpsfiguren maken de plek tastbaar; "
                "stel u bij een brug of erf een kort verhaal voor.")
    if cat == "architectuur_route":
        return ("Architectuur langs de route. Let op bruggetjes, sluizen, beschoeiingen en hekwerken direct langs het pad; "
                "materialen en bevestigingen verraden leeftijd en onderhoud.")
    return "Observeer rustig en blijf alert op het pad."



## D) Generate POIs and write SSML & CSV


In [12]:

targets = np.linspace(total_m * 0.07, total_m * 0.93, POI_COUNT)
idxs = [int(np.argmin([abs(d - t) for d in cum])) for t in targets]

pois=[]
t0 = next((t for _,_,t in pts if t is not None), None)
for i, idx in enumerate(idxs):
    lat, lon, t = pts[idx]
    km = round(cum[idx]/1000.0, 2)
    # ETA
    if t is not None:
        eta = t
    else:
        start = t0 if t0 else datetime(1970,1,1, tzinfo=timezone.utc)
        hours = (cum[idx]/1000.0)/PACE_KMH
        eta = (start + timedelta(hours=hours))
    cat = PREFS["categories"][i % len(PREFS["categories"])]
    txt = text_for(cat)
    ssml_file = OUTDIR/"ssml"/f"POI-{i+1:02d}_{cat}.ssml"
    ssml_file.write_text(ssml(txt), encoding="utf-8")
    pois.append({
        "id":f"POI-{i+1:02d}","km":km,"lat":lat,"lon":lon,"eta_utc":eta.astimezone(timezone.utc).isoformat(),
        "duration_s":100 if i%2==0 else 95,"category":cat,
        "persona":{"landschap":"Ecologist","geschiedenis_ijzertijd":"Historian","cultuur_bekende_personen":"CulturalGuide","architectuur_route":"Architect"}[cat],
        "title": txt.split(".")[0],
        "ssml_file": f"ssml/{ssml_file.name}"
    })

# CSV
with (OUTDIR/"pois.csv").open("w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["id","km","lat","lon","eta_utc","duration_s","category","persona","title","ssml_file"])
    for p in pois: w.writerow([p[k] for k in ["id","km","lat","lon","eta_utc","duration_s","category","persona","title","ssml_file"]])

# Intro/Outro
(OUTDIR/"ssml"/"intro.ssml").write_text(ssml("Welkom bij deze wandeling. Rustig tempo, kalme verhalen; we starten met een korte warming-up."), encoding="utf-8")
(OUTDIR/"ssml"/"outro.ssml").write_text(ssml("Dank voor het meewandelen. We sluiten af met rustige stretches. Tot de volgende keer."), encoding="utf-8")

# Copy GPX
(OUTDIR/"gpx"/GPX_PATH.name).write_text(Path(GPX_PATH).read_text(encoding="utf-8"), encoding="utf-8")

print("POIs:", len(pois))


POIs: 12



## E) Music suggestions, language deck, brain snacks, stretches


In [13]:

# Music (80s, suggestions only)
suggestions = [
    ["a-ha – Take On Me (3:48)", "Eurythmics – Sweet Dreams (3:36)"],
    ["Toto – Africa (4:55)", "Simple Minds – Don't You (4:20)"],
    ["The Cure – Just Like Heaven (3:32)", "New Order – Blue Monday (3:43)"]
]
music_cues = []
for i, start_min in enumerate([5, max(10, duration_min//2 - 5), max(15, duration_min - 25)]):
    block = suggestions[i]
    dur_s = sum([int(t.split("(")[-1].replace(")","").split(":")[0])*60 + int(t.split("(")[-1].replace(")","").split(":")[1]) for t in block])
    music_cues.append({"start_min": start_min, "duration_s": dur_s, "suggestions": block})
(OUTDIR/"cues"/"music_suggestions.json").write_text(json.dumps(music_cues, indent=2, ensure_ascii=False), encoding="utf-8")

# Language (Norwegian A1 deck)
with (OUTDIR/"lang"/"deck_no_A1.csv").open("w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["index","norwegian","dutch","example"])
    vocab = [
        ("hei","hallo"),("takk","dank je"),("vær så snill","alsjeblieft (beleefd)"),("unnskyld","sorry/excuseer"),
        ("god morgen","goedemorgen"),("god kveld","goedenavond"),("ja","ja"),("nei","nee"),("kanskje","misschien"),
        ("hvor","waar"),("hvem","wie"),("hva","wat"),("når","wanneer"),("hvorfor","waarom"),("hvordan","hoe"),
        ("vann","water"),("mat","eten"),("hjelp","help"),("toalett","toilet"),("billett","kaartje/ticket")
    ]
    sentence = ("Kan du hjelpe meg? Jeg leter etter busstasjonen.","Kunt u mij helpen? Ik ben op zoek naar het busstation.")
    for i,(no,nl) in enumerate(vocab, start=1):
        ex = "Jeg trenger hjelp." if no=="hjelp" else ""
        w.writerow([i,no,nl,ex])
    w.writerow([21, sentence[0], sentence[1], ""])

# Brain
total_km = total_m/1000.0
math_snacks = [
    {"at_km": round(total_km*0.15,2), "type":"add", "prompt":"Tel op: 27 + 18"},
    {"at_km": round(total_km*0.35,2), "type":"sub", "prompt":"Reken uit: 85 − 27"},
    {"at_km": round(total_km*0.55,2), "type":"mul", "prompt":"Wat is 7 × 6?"},
    {"at_km": round(total_km*0.75,2), "type":"div", "prompt":"Wat is 48 ÷ 6?"}
]
memory_snacks = [
    {"at_km": round(total_km*0.25,2), "type":"recall", "prompt":"Noors: herhaal ‘hei, takk, vann’."},
    {"at_km": round(total_km*0.65,2), "type":"nback", "prompt":"Onthoud de reeks: rood, boom, 7. Herhaal na 20 seconden."}
]
(OUTDIR/"brain"/"math_snacks.json").write_text(json.dumps(math_snacks, indent=2, ensure_ascii=False), encoding="utf-8")
(OUTDIR/"brain"/"memory_snacks.json").write_text(json.dumps(memory_snacks, indent=2, ensure_ascii=False), encoding="utf-8")

# Stretches
(OUTDIR/"cues"/"stretches.json").write_text(json.dumps([{"at_min":0,"type":"warmup"},{"at_min":max(5, duration_min-12),"type":"cooldown"}], indent=2, ensure_ascii=False), encoding="utf-8")
print("Cues, language, brain, stretches written.")


Cues, language, brain, stretches written.



## F) Build manifest.json


In [14]:

manifest = {
    "routeId": GPX_PATH.stem,
    "version": "1.0.0",
    "locale": "nl-NL",
    "pace_kmh": PACE_KMH,
    "duration_min": duration_min,
    "gpx": f"gpx/{GPX_PATH.name}",
    "user_prefs": PREFS,
    "segments": [
        {
            "id": p["id"],
            "geo": {"lat": p["lat"], "lon": p["lon"], "radius_m": 60},
            "layers": [{
                "category": p["category"],
                "persona": p["persona"],
                "depth": "overview",
                "duration_s": p["duration_s"],
                "title": p["title"],
                "ssml": p["ssml_file"],
                "sources": []
            }],
            "fallbacks": {"language_snack": "lang/deck_no_A1.csv"}
        } for p in pois
    ],
    "cues": {
        "music": json.loads((OUTDIR/'cues'/'music_suggestions.json').read_text(encoding='utf-8')),
        "stretches": json.loads((OUTDIR/'cues'/'stretches.json').read_text(encoding='utf-8'))
    }
}
(OUTDIR/"manifest.json").write_text(json.dumps(manifest, indent=2, ensure_ascii=False), encoding="utf-8")
print("Manifest written:", (OUTDIR/'manifest.json').resolve())


Manifest written: C:\Development\Python\RoamRhythm\pack_from_notebook\manifest.json



## G) Preview POIs (table)


In [15]:

import pandas as pd
pd.DataFrame([{k:v for k,v in p.items() if k != 'ssml'} for p in pois])


,id,km,lat,lon,eta_utc,duration_s,category,persona,title,ssml_file
0,POI-01,0.71,51.592805,5.158061,1970-01-01T00:10:36.346533+00:00,100,landschap,Ecologist,Landschap & waterstaat,ssml/POI-01_landschap.ssml
1,POI-02,1.50,51.591418,5.167431,1970-01-01T00:22:28.995552+00:00,95,geschiedenis_ijzertijd,Historian,Geschiedenis — focus op de IJzertijd,ssml/POI-02_geschiedenis_ijzertijd.ssml
2,POI-03,2.29,51.596083,5.172364,1970-01-01T00:34:24.546355+00:00,100,cultuur_bekende_personen,CulturalGuide,Cultuur & lokale figuren,ssml/POI-03_cultuur_bekende_personen.ssml
3,POI-04,3.09,51.601088,5.180540,1970-01-01T00:46:20.587609+00:00,95,architectuur_route,Architect,Architectuur langs de route,ssml/POI-04_architectuur_route.ssml
4,POI-05,3.88,51.602882,5.177724,1970-01-01T00:58:15.169491+00:00,100,landschap,Ecologist,Landschap & waterstaat,ssml/POI-05_landschap.ssml
5,POI-06,4.68,51.602904,5.167198,1970-01-01T01:10:09.530210+00:00,95,geschiedenis_ijzertijd,Historian,Geschiedenis — focus op de IJzertijd,ssml/POI-06_geschiedenis_ijzertijd.ssml
6,POI-07,5.47,51.605731,5.157479,1970-01-01T01:22:06.202597+00:00,100,cultuur_bekende_personen,CulturalGuide,Cultuur & lokale figuren,ssml/POI-07_cultuur_bekende_personen.ssml
7,POI-08,6.26,51.609399,5.147808,1970-01-01T01:33:52.774345+00:00,95,architectuur_route,Architect,Architectuur langs de route,ssml/POI-08_architectuur_route.ssml
8,POI-09,7.05,51.609925,5.139247,1970-01-01T01:45:46.517623+00:00,100,landschap,Ecologist,Landschap & waterstaat,ssml/POI-09_landschap.ssml
9,POI-10,7.85,51.604854,5.140598,1970-01-01T01:57:41.703250+00:00,95,geschiedenis_ijzertijd,Historian,Geschiedenis — focus op de IJzertijd,ssml/POI-10_geschiedenis_ijzertijd.ssml



## H) One-click: ZIP the pack for download


In [16]:

import zipfile
zip_path = OUTDIR.with_suffix(".zip")
with zipfile.ZipFile(zip_path, "w") as z:
    for p in OUTDIR.rglob("*"):
        z.write(p, p.relative_to(OUTDIR.parent))
print("ZIP:", zip_path.resolve())


ZIP: C:\Development\Python\RoamRhythm\pack_from_notebook.zip
